In [1]:
%load_ext autotime

In [13]:
import os

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook as tqdm

from piaa.utils import pipeline
from piaa.utils import helpers

from pocs.utils.images import fits as fits_utils

from astropy.stats import sigma_clip, sigma_clipped_stats
from astropy import units as u

#### Observation sources

We use the previously detected and filtered sources to examine our target. See the [Source-Detection](Source-Detection-And-Filtering.ipynb) for details.

In [2]:
base_dir = '/var/panoptes/images/fields/PAN012/358d0f/20180822T035809/'
# base_dir = '/var/panoptes/images/fields/PAN001/Hd189733/14d3bd/20180913T085704/'
source_filename = os.path.join(base_dir, f'point-sources-filtered.csv')

sources = pipeline.lookup_sources_for_observation(filename=source_filename).set_index(['picid'], append=True)

For our target table we want to compare the flux as calculated from `sextractor` (which has no knowledge of the RGB array) with that pulled from a custom stamp.  `sextractor` uses a 6-pixel circular aperture  centered around the calculated centroid (the `x` and `y` values in our `sources` table) while our custom stamps will use an aperture that is sliced according to our rule of having a fixed pixel pattern. Here we use a 6-pixel aperture on our stamps for direct comparison with `sextractor` but the size can be vary.

In [6]:
camera_bias = 2048 * u.adu

readout_noise = 10.5 * u.electron / u.pixel
gain = 1.5 * (u.electron / u.adu)

stamp_size = 10 * u.pixel
num_pixels = (stamp_size**2).value

# Lookup exposure time used
exptime = fits_utils.getval(os.path.join(base_dir, sources.iloc[0].file), 'EXPTIME')

num_sources = len(sources.index.levels[1])

In [14]:
counts = list()

for pid, target_table in tqdm(sources.groupby('picid'), total=num_sources): 
    # Get PSC info for this target
    psc = pd.read_csv(os.path.join(base_dir, 'stamps', f'{pid}.csv'), parse_dates=True).set_index(['obs_time', 'picid'])
    
    # Loop through each frame of the PSC
    for idx, row in psc.iterrows():
        date_obs= idx[0]
        row_info = dict(date_obs=date_obs, picid=pid)
        
        # Get the counts with bias removed
        stamp_counts = row.values - camera_bias.value
        stamp_electrons = stamp_counts * gain.value
        
        back_mean, back_median, back_std = sigma_clipped_stats(stamp_electrons)
        back_mean_counts = back_mean / gain.value
        back_std_counts = back_std / gain.value
        
        # Store the overall flux
        count_sum = (stamp_counts - back_mean_counts).sum()
        count_noise = np.sqrt(count_sum)
        
        # Readout noise
        readout = ((readout_noise) * num_pixels).value
        readout = readout / gain.value    
        
        # Dark noise (see Zhang et al 2016)
        dark_noise = (0.1 * exptime * num_pixels) / gain.value   
        
        counts.append({
            'date_obs': date_obs,
            'picid': pid,            
            'channel': 'all',
            'counts': count_sum,
            'counts_noise': count_noise,
            'back': back_mean_counts,
            'back_noise': back_std_counts,
            'readout_noise': readout,
            'dark_noise': dark_noise,
        })
        
        color_data = helpers.get_rgb_data(stamp_electrons.reshape(int(stamp_size.value), int(stamp_size.value)), force_new=True)

        # Calculate some properties for each channel
        for color, color_electrons in zip('rgb', color_data):
            
            back_mean, back_median, back_std = sigma_clipped_stats(color_electrons)
            back_mean_counts = back_mean / gain.value
            back_std_counts = back_std / gain.value
            
            color_counts = color_electrons / gain.value
            count_sum = (color_counts - back_mean_counts).sum()
            count_noise = np.sqrt(count_sum)
            
            if count_sum <= 0:
#                 print(f'Negative stamp: {pid} {color} {date_obs} {stamp_sum}')
                continue
                
            n_stamp_pixels = int(color_electrons.count())
                        
            # Readout noise
            readout = ((readout_noise) * n_stamp_pixels).value
            readout = readout / gain.value    

            # Dark noise (see Zhang et al 2016)
            dark_noise = (0.1 * exptime * n_stamp_pixels) / gain.value

            counts.append({
                'date_obs': date_obs,
                'picid': pid,
                'channel': color,
                'counts': count_sum,
                'counts_noise': count_noise,
                'back': back_mean_counts,
                'back_noise': back_std_counts,
                'readout_noise': readout,
                'dark_noise': dark_noise,
            })            

source_noise = pd.DataFrame(counts).set_index(['date_obs'])
source_noise.index = pd.to_datetime(source_noise.index)
source_noise.to_csv(os.path.join(base_dir, f'point-sources-noise.csv'))

/home/wtylergee/miniconda3/envs/pipeline/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in sqrt
/home/wtylergee/miniconda3/envs/pipeline/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in sqrt
